<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-06 12:16:59
-------------------
qualified stocks: 93
with latest results: 31
still star stocks: 17


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.64 L
Current:  1.33 C
-------------------
Today PnL: 83.82 K (0.63%)
Current PnL: -15.52 L (-10.81%)
CY Booked + Current PnL: -10.52 L (-7.33%)
-------------------
Total profit:  4.57 L
Total loss:  -20.09 L
-------------------
Total Booked + Current PnL: 16.46 L (14.12%)
Total Booked PnL: 31.97 L (27.43%)
Curr Year Booked PnL: 4.99 L (3.75%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 78.40 L (58.9%)
-------------------
Est LTT:  2.74 C
Est LTT PnL: 1.41 C (105.64%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DLF,714.31,885.0,-5.13,H-LC,92.47,289162.0,52725.0,3759.0,5.80,22.30,1.30,23.90,30.0,14.03,2.20,43.67,AR,NTT,REALTY
50,LICI,785.05,983.0,-3.65,H-LC,89.25,237187.0,42495.0,6594.0,-0.02,21.83,2.78,25.21,31.0,6.44,1.80,31.16,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,62.37,104462.0,26100.0,7929.0,1.00,33.31,7.59,43.43,70.0,3.29,0.79,39.72,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,6.12,X-LC,25.81,157527.0,16718.0,8806.0,0.19,11.87,5.59,18.13,19.0,1.90,1.20,24.38,XY25,NTT,REFINERIES
85,UNIONBANK,123.87,163.0,-0.31,M-LC,93.55,174882.0,34042.0,10440.0,0.83,24.17,5.97,31.59,79.0,3.26,1.33,51.91,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,60.00,49.36,M-SC,82.80,129303.0,-13176.0,32701.0,7.62,-9.25,25.29,13.70,221.0,-0.40,0.98,52.61,X40N,NTT,BANKS
26,DLF,714.31,885.00,-5.13,H-LC,92.47,289162.0,52725.0,3759.0,5.80,22.30,1.30,23.90,30.0,14.03,2.20,43.67,AR,NTT,REALTY
60,RBLBANK,230.61,388.00,42.75,M-SC,60.22,222087.0,-12213.0,172117.0,5.65,-5.21,77.50,68.25,222.0,-0.07,1.69,46.25,XY24,NTT,BANKS
64,ROUTE,1558.40,2249.26,-38.28,M-SC,40.86,66396.0,-36458.0,82052.0,4.38,-35.45,123.58,44.33,172.0,-0.44,0.51,10.26,SR,ATH,IT
63,REPCOHOME,515.07,880.00,-55.29,H-SC,58.06,165170.0,-39828.0,185073.0,4.09,-19.43,112.05,70.85,121.0,-0.22,1.26,33.33,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-365.75,H-SC,100.00,112970.0,-20316.0,104915.0,-2.43,-15.24,92.87,63.47,130.0,-0.19,0.86,89.37,XR,NTT,CHEMICALS
73,SONACOMS,578.05,1006.0,-28.54,M-MC,35.48,92418.0,-8741.0,83629.0,-1.85,-8.64,90.49,74.03,155.0,-0.10,0.70,29.88,AR,BTT,AUTO
56,PFIZER,4245.50,6318.3,13.55,H-SC,97.85,131272.0,33626.0,14046.0,-1.54,34.44,10.70,48.82,134.0,2.39,1.00,45.83,X40,ATH,PHARMA
22,COFFEEDAY,59.14,80.0,-61.77,L-SC,6.45,57408.0,-56141.0,96193.0,-1.52,-49.44,167.56,35.27,239.0,-0.58,0.44,40.51,XR,NTT,HOTELS
14,BERGEPAINT,558.54,680.0,-14.92,H-MC,43.01,182481.0,5424.0,33084.0,-1.09,3.06,18.13,21.75,104.0,0.16,1.39,30.53,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-13.14,X-LC,37.63,278882.0,17281.0,128341.0,0.24,6.61,46.02,55.67,1.0,0.13,2.12,13.11,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.37,X-LC,13.98,263203.0,-32731.0,126653.0,0.10,-11.06,48.12,31.73,2.0,-0.26,2.00,5.35,X40,BTT,IT
51,LTIM,5564.16,7197.33,-3.08,X-LC,73.12,186768.0,-13542.0,72335.0,0.13,-6.76,38.73,29.35,3.0,-0.19,1.42,29.03,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.26,X-LC,4.30,152551.0,-48835.0,136442.0,-0.00,-24.25,89.44,43.51,5.0,-0.36,1.16,5.22,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-15.17,X-LC,17.20,266448.0,-3899.0,38528.0,0.11,-1.44,14.46,12.81,6.0,-0.10,2.03,10.27,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4245.50,6318.30,13.55,H-SC,97.85,131272.0,33626.0,14046.0,-1.54,34.44,10.70,48.82,134.0,2.39,1.00,45.83,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.69,H-MC,27.96,102298.0,17197.0,25902.0,0.62,20.21,25.32,50.65,90.0,0.66,0.78,28.59,X40N,ATH,CABLES
85,UNIONBANK,123.87,163.00,-0.31,M-LC,93.55,174882.0,34042.0,10440.0,0.83,24.17,5.97,31.59,79.0,3.26,1.33,51.91,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,62.37,104462.0,26100.0,7929.0,1.00,33.31,7.59,43.43,70.0,3.29,0.79,39.72,XY24,NTT,MISC
50,LICI,785.05,983.00,-3.65,H-LC,89.25,237187.0,42495.0,6594.0,-0.02,21.83,2.78,25.21,31.0,6.44,1.80,31.16,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,102.64,M-MC,88.17,230318.0,5356.0,162305.0,0.48,2.38,70.47,74.53,170.0,0.03,1.75,33.72,XY24,BTT,STEEL
52,LUXIND,1448.19,2442.00,38.42,M-SC,68.82,143650.0,6072.0,88345.0,-0.99,4.41,61.50,68.62,183.0,0.07,1.09,24.53,X40N,NTT,TEXTILES
56,PFIZER,4245.50,6318.30,13.55,H-SC,97.85,131272.0,33626.0,14046.0,-1.54,34.44,10.70,48.82,134.0,2.39,1.00,45.83,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.69,H-MC,27.96,102298.0,17197.0,25902.0,0.62,20.21,25.32,50.65,90.0,0.66,0.78,28.59,X40N,ATH,CABLES
18,CAMS,3643.00,5250.99,6.44,H-SC,86.02,119560.0,17556.0,27463.0,0.51,17.21,22.97,44.14,112.0,0.64,0.91,36.25,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,6.44,H-SC,86.02,119560.0,17556.0,27463.0,0.51,17.21,22.97,44.14,112.0,0.64,0.91,36.25,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-28.22,M-MC,70.97,141601.0,12449.0,74624.0,0.14,9.64,52.70,67.42,156.0,0.17,1.08,37.72,XR,NTT,BANKS
39,INDIAMART,2327.09,4850.92,-51.09,H-SC,55.91,127544.0,4208.0,129559.0,-0.05,3.41,101.58,108.45,108.0,0.03,0.97,26.53,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.16,M-LC,33.33,154076.0,3131.0,106328.0,0.23,2.07,69.01,72.51,68.0,0.03,1.17,9.96,XR,NTT,IT
1,AARTIIND,487.04,919.00,38.61,M-SC,53.76,79651.0,-1685.0,73821.0,-0.98,-2.07,92.68,88.69,205.0,-0.02,0.61,34.31,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.5,-22.76,H-SC,64.52,87900.0,-10080.0,31952.0,2.92,-10.29,36.35,22.32,127.0,-0.32,0.67,27.49,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,62.37,104462.0,26100.0,7929.0,1.00,33.31,7.59,43.43,70.0,3.29,0.79,39.72,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-365.75,H-SC,100.00,112970.0,-20316.0,104915.0,-2.43,-15.24,92.87,63.47,130.0,-0.19,0.86,89.37,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,49.36,M-SC,82.80,129303.0,-13176.0,32701.0,7.62,-9.25,25.29,13.70,221.0,-0.40,0.98,52.61,X40N,NTT,BANKS
56,PFIZER,4245.50,6318.3,13.55,H-SC,97.85,131272.0,33626.0,14046.0,-1.54,34.44,10.70,48.82,134.0,2.39,1.00,45.83,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-365.75,H-SC,100.00,112970.0,-20316.0,104915.0,-2.43,-15.24,92.87,63.47,130.0,-0.19,0.86,89.37,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.00,6627.78,M-SC,98.92,73844.0,-19966.0,96588.0,3.45,-21.28,130.80,81.67,214.0,-0.21,0.56,43.00,XR,NTT,CERAMICS
56,PFIZER,4245.50,6318.30,13.55,H-SC,97.85,131272.0,33626.0,14046.0,-1.54,34.44,10.70,48.82,134.0,2.39,1.00,45.83,X40,ATH,PHARMA
81,TITAGARH,1097.23,1548.00,-6.89,H-SC,96.77,163625.0,-28390.0,107273.0,0.11,-14.79,65.56,41.08,128.0,-0.26,1.24,38.49,XY24,NTT,ENGINEERING
77,TANLA,917.30,1963.11,-33.39,H-SC,95.70,163112.0,-76303.0,349255.0,-0.58,-31.87,214.12,114.01,113.0,-0.22,1.24,49.60,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.77
1,25,42.20
2,50,70.58


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.68
AR      15.15
XY25    14.57
X40N    13.25
X40     12.46
XR      10.32
X200     1.42
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.03
X-LC    17.99
M-SC    17.52
H-LC     9.29
H-MC     8.51
M-LC     7.71
M-MC     5.50
X-MC     5.48
X-SC     2.75
L-SC     2.29
L-LC     1.12
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.39,-1.79,33.65
IT,10.17,-16.70,80.09
BANKS,9.81,-8.65,58.60
MISC,6.49,-26.56,82.17
FINANCE,5.91,-23.15,81.63
INSURANCE,5.28,3.94,24.76
PAINTS,5.27,-18.68,53.81
ELECTRICAL,5.11,-4.54,49.89
HEALTHCARE,4.34,-4.22,35.32


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2808571.0,26
AR,1230057.0,14
X40N,1203626.0,17
XR,1097759.0,14
X40,770585.0,9
XY25,462097.0,10
SR,194710.0,2
X200,72335.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2206489.0,20
M-SC,2011563.0,21
X-LC,847203.0,11
M-MC,543708.0,5
H-MC,507587.0,8
X-SC,348310.0,4
X-MC,311598.0,5
L-SC,298567.0,4
M-LC,290429.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      913706.0      6
M-SC       XY24      745139.0      6
H-SC       AR        538329.0      3
           X40N      500633.0      6
X-LC       X40       479143.0      4
M-SC       X40N      463572.0      6
M-MC       XY24      385455.0      3
M-SC       XR        288530.0      4
H-SC       XR        239775.0      4
H-LC       AR        223879.0      5
M-SC       X40       218434.0      2
H-MC       XY24      173672.0      2
M-LC       XY24      171278.0      4
X-LC       AR        166219.0      2
L-SC       XR        159310.0      2
X-MC       XY24      140044.0      2
X-LC       XY25      129506.0      4
X-SC       X40N      119603.0      2
M-SC       XY25      119551.0      1
X-SC       XR        116049.0      1
H-MC       XR        113143.0      1
X-SC       SR        112658.0      1
M-LC       XR        106328.0      1
M-SC       AR         94285.0      1
X-MC       X40N       93916.0      2
M-MC       AR         83629.0      1
M-SC       SR         82052.0      1
L-SC       XY24       77847.0      1
X-MC       XY25       77638.0      1
M-MC       XR         74624.0      1
H-MC       XY25       73602.0      1
X-LC       X200       72335.0      1
H-MC       AR         62306.0      1
L-SC       AR         61410.0      1
H-MC       X40        58962.0      2
L-LC       XY25       42383.0      1
H-MC       X40N       25902.0      1
H-SC       X40        14046.0      1
M-LC       XY25       12823.0      1
H-LC       XY25        6594.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 44.0 seconds
